Import Modules

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train= False,
    download=True,
    transform=ToTensor(),
)


In [18]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") # N = Count, C= Class, W= width H = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")   #     
    break
    # print(f"index {y.size()}")  # 64 개
    #print(X)
    # print(y)

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Tensor y: torch.Size([64]) torch.int64


In [19]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To train a model, we need a loss function and an optimizer.

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [26]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 1.047032  [    0/60000]
loss: 1.071133  [ 6400/60000]
loss: 0.865821  [12800/60000]
loss: 1.019421  [19200/60000]
loss: 0.907483  [25600/60000]
loss: 0.926579  [32000/60000]
loss: 0.987020  [38400/60000]
loss: 0.925177  [44800/60000]
loss: 0.958815  [51200/60000]
loss: 0.906713  [57600/60000]
Test Error: 
 Accuracy: 67.0%, Avg loss: 0.913835 

Epoch 2
-------------------------------
loss: 0.957434  [    0/60000]
loss: 1.003494  [ 6400/60000]
loss: 0.782618  [12800/60000]
loss: 0.952434  [19200/60000]
loss: 0.847407  [25600/60000]
loss: 0.855879  [32000/60000]
loss: 0.931111  [38400/60000]
loss: 0.873040  [44800/60000]
loss: 0.900494  [51200/60000]
loss: 0.855312  [57600/60000]
Test Error: 
 Accuracy: 68.1%, Avg loss: 0.860393 

Epoch 3
-------------------------------
loss: 0.889311  [    0/60000]
loss: 0.952994  [ 6400/60000]
loss: 0.720843  [12800/60000]
loss: 0.902889  [19200/60000]
loss: 0.805049  [25600/60000]
loss: 0.803075  [32000/600

In [27]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [28]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [32]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    pred = model(x)
    print(f'{pred} {pred.shape}')    
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([[ 2.6528, -4.3668,  7.7429, -0.5734,  5.8013, -2.9029,  6.2302, -9.6228,
          2.4913, -4.4411]]) torch.Size([1, 10])
Predicted: "Pullover", Actual: "Pullover"
